In [1]:
#@title **Install Conda Colab**
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


In [1]:
%%capture
!pip install git+https://github.com/pablo-arantes/biopandas
!mamba install openmmforcefields -c conda-forge -y
!pip install openMiChroM

In [2]:
from OpenMiChroM.ChromDynamics import MiChroM
from OpenMiChroM.CndbTools import cndbTools
import numpy as np

In [145]:
sim = MiChroM(temperature=1.0, time_step=0.01)

    ***************************************************************************************     
     **** **** *** *** *** *** *** *** OpenMiChroM-1.0.7 *** *** *** *** *** *** **** ****      

         OpenMiChroM is a Python library for performing chromatin dynamics simulations.         
                            OpenMiChroM uses the OpenMM Python API,                             
                employing the MiChroM (Minimal Chromatin Model) energy function.                
      The chromatin dynamics simulations generate an ensemble of 3D chromosomal structures      
      that are consistent with experimental Hi-C maps, also allows simulations of a single      
                 or multiple chromosome chain using High-Performance Computing                  
                            in different platforms (GPUs and CPUs).                             
         OpenMiChroM documentation is available at https://open-michrom.readthedocs.io          

         OpenMiChroM is desc

In [146]:
sim.setup(platform="cuda")

In [147]:
sim.saveFolder('17output_chr2')

In [148]:
chr2 = sim.createSpringSpiral(ChromSeq='/content/chr2_compartments.txt', isRing=False)

In [149]:
sim.loadStructure(chr2, center=True)

In [150]:
sim.saveStructure(mode='ndb')

In [151]:
sim.addFENEBonds(kfb=30.0)
sim.addAngles(ka=2.0)
sim.addRepulsiveSoftCore(Ecut=4.0)

In [152]:
def AM_bond(sim, pair_list, depth, sigma=1):
    R"""
    Internal function that inits FM bond force.
    """
    bondforceGr = sim.mm.CustomBondForce(f'-depth_w*exp((r-r0)^2/(-2.0*sigma^2))')
    bondforceGr.addGlobalParameter('depth_w', depth)
    bondforceGr.addPerBondParameter('r0')
    bondforceGr.addPerBondParameter('sigma')

    sim.forceDict["AM"] = bondforceGr

    for k  in range(len(pair_list)):
        i, j, ro = pair_list[k]
        sim.forceDict["AM"].addBond(int(i), int(j), [ro,np.sqrt(j-i)*sigma])

In [153]:
pair_list=np.loadtxt('/content/pairs_chr2_h1.txt')
pair_list[:,2]=np.round(pair_list[:,2],4)
print('Number of constrains:',len(pair_list))

Number of constrains: 436645


In [154]:
AM_bond(sim,pair_list,5.0,sigma=2)

In [155]:
sim.addFlatBottomHarmonic(kr=5*10**-3, n_rad=15.0)

In [156]:
block = 3*10**2
n_blocks = 2*10**3

In [157]:
for _ in range(n_blocks):
    sim.runSimBlock(block, increment=False)

Number of exceptions: 4843
adding force  FENEBond 0
adding force  AngleForce 1
Add exclusions for RepulsiveSoftCore force
adding force  RepulsiveSoftCore 2
adding force  AM 3
adding force  FlatBottomHarmonic 4
Positions... 
 loaded!
potential energy is 170.029611
bl=0 pos[1]=[374.2 -19.5 -2.6] dr=0.76 t=3.0ps kin=1.73 pot=170.01 Rg=262.213 SPS=2079 
bl=0 pos[1]=[371.2 -18.8 -0.6] dr=1.64 t=6.0ps kin=3.20 pot=167.99 Rg=260.694 SPS=2973 
bl=0 pos[1]=[367.8 -17.5 0.8] dr=2.30 t=9.0ps kin=4.67 pot=165.07 Rg=258.502 SPS=3833 
bl=0 (i) pos[1]=[363.8 -17.7 2.6] dr=2.79 t=12.0ps kin=5.91 pot=161.53 Rg=255.821 SPS=3858 
bl=0 pos[1]=[363.7 -17.5 3.5] dr=0.86 t=15.0ps kin=2.03 pot=160.95 Rg=255.286 SPS=4300 
bl=0 pos[1]=[362.8 -17.2 5.3] dr=1.60 t=18.0ps kin=3.25 pot=159.08 Rg=253.845 SPS=4023 
bl=0 pos[1]=[361.0 -16.7 6.6] dr=2.21 t=21.0ps kin=4.56 pot=156.35 Rg=251.754 SPS=3618 
bl=0 (i) pos[1]=[357.4 -15.9 7.1] dr=2.67 t=24.0ps kin=5.63 pot=153.10 Rg=249.195 SPS=4327 
bl=0 pos[1]=[355.6 -16.7 

In [158]:
print(sim.chromRG())
sim.saveStructure(mode='ndb')

10.953816


In [159]:
sim.saveStructure(mode='gro')
sim.saveStructure(mode='pdb')

In [160]:
sim.removeFlatBottomHarmonic()

Removed FlatBottomHarmonic from the system!


In [161]:
sim.forceDict.keys()

dict_keys(['FENEBond', 'AngleForce', 'RepulsiveSoftCore', 'AM'])

In [162]:
sim.initStorage(filename="traj_chr10")

In [163]:
block = 5*10**2
n_blocks = 5*10**3

In [164]:
for _ in range(n_blocks):
    sim.runSimBlock(block, increment=True)
    sim.saveStructure()

Streaming output truncated to the last 5000 lines.
bl=1634 pos[1]=[3.2 7.4 7.6] dr=1.03 t=14170.0ps kin=1.53 pot=-428.11 Rg=10.878 SPS=3734 
bl=1635 pos[1]=[3.2 7.1 7.7] dr=1.05 t=14175.0ps kin=1.49 pot=-428.07 Rg=10.886 SPS=3696 
bl=1636 pos[1]=[3.3 6.9 7.1] dr=1.06 t=14180.0ps kin=1.50 pot=-428.10 Rg=10.889 SPS=3637 
bl=1637 pos[1]=[2.7 7.1 6.8] dr=1.06 t=14185.0ps kin=1.51 pot=-428.07 Rg=10.910 SPS=3702 
bl=1638 pos[1]=[3.4 5.6 6.1] dr=1.05 t=14190.0ps kin=1.52 pot=-428.07 Rg=10.932 SPS=3636 
bl=1639 pos[1]=[3.5 6.5 6.6] dr=1.06 t=14195.0ps kin=1.52 pot=-428.07 Rg=10.938 SPS=3752 
bl=1640 pos[1]=[4.3 5.8 7.8] dr=1.05 t=14200.0ps kin=1.52 pot=-428.09 Rg=10.897 SPS=3727 
bl=1641 pos[1]=[4.6 5.7 7.4] dr=1.05 t=14205.0ps kin=1.49 pot=-428.07 Rg=10.893 SPS=3739 
bl=1642 pos[1]=[3.3 5.9 6.6] dr=1.03 t=14210.0ps kin=1.47 pot=-428.07 Rg=10.918 SPS=3660 
bl=1643 pos[1]=[2.2 5.7 7.0] dr=1.02 t=14215.0ps kin=1.50 pot=-428.06 Rg=10.937 SPS=3628 
bl=1644 pos[1]=[3.3 6.4 8.2] dr=1.04 t=14220.0ps 

In [165]:
sim.storage[0].close()

In [166]:
sim.saveStructure(mode='pdb')